In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("..")  # Go up one level to the UROP directory

In [2]:
import yaml

with open("config.yaml") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

batch_size = config["batch_size"]
lr = config["lr"]
num_epochs = config["num_epochs"]
save_every = config["save_every"]
save_dir = config["save_dir"]
num_workers = config["num_workers"]

In [ ]:
import torch
from torch.utils.data import DataLoader
from src import LazyWindowedDataset

test_source_dataset = LazyWindowedDataset(
    root_dir="datasets/RoboticArm",
    split="test",
    anomaly_type=['normal', 'anomaly'],
    domain_type=['source', 'target'],
    window_size_ms=100,
    stride_ms=50,
)
test_loader = DataLoader(test_source_dataset, batch_size=batch_size, shuffle=True, num_workers=1, drop_last=True)

In [ ]:
from models import BaselineAutoencoder
from src import evaluate, load_model_checkpoint

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model, _, _, _, _, _ = load_model_checkpoint("checkpoints/RoboticArmBaseline.pt", BaselineAutoencoder, optimizer_class=None)
criterion = torch.nn.MSELoss()

![alt text](figures/roboticArmHighlightedTable.png)

In [5]:
loss, auc = evaluate(model, test_loader, criterion)
print(f"Overall S+T | Loss: {loss:.4f}, AUC: {auc:.4f}")

Overall S+T | Loss: 0.6323, AUC: 0.8988


In [ ]:
acc_loss, acc_auc = evaluate(model, test_loader, criterion, sensors_to_test=['acc'])
print(f"Acc S+T | Loss: {acc_loss:.4f}, AUC: {acc_auc:.4f}")

Acc s+T | Loss: 0.1346, AUC: 0.8576


In [7]:
gyro_loss, gyro_auc = evaluate(model, test_loader, criterion, sensors_to_test=['gyro'])
print(f"Gyro S+T | Loss: {gyro_loss:.4f}, AUC: {gyro_auc:.4f}")

Gyro S+T | Loss: 1.4949, AUC: 0.6337


In [8]:
mic_loss, mic_auc = evaluate(model, test_loader, criterion, sensors_to_test=['mic'])
print(f"Mic S+T | Loss: {mic_loss:.4f}, AUC: {mic_auc:.4f}")

Mic S+T | Loss: 1.0299, AUC: 0.5433
